In [1]:
from gevent import monkey
monkey.patch_all(thread=False)
from gevent.queue import Queue
import gevent

# = = = = = = = = = = = = = = =

import pandas as pd

output_correct = pd.DataFrame()
output_error = pd.DataFrame()

# = = = = = = = = = = = = = = =

input_ = pd.read_excel('./item.xlsx',
                       header=0,
                       dtype=str).fillna('')

print('总数量：' + str(len(input_)))
print()

# = = = = = = = = = = = = = = =

import requests

import sys
sys.path.append('../../00.Tools')
import UA
import Proxy

# = = = = = = = = = = = = = = =

from bs4 import BeautifulSoup
from lxml import etree

# = = = = = = = = = = = = = = =

import re

# = = = = = = = = = = = = = = =

import time
from datetime import datetime

# = = = = = = = = = = = = = = =

work = Queue()
for a in range(len(input_)):
    work.put_nowait(a)
    
# = = = = = = = = = = = = = = =

def crawler():
    global output_correct, output_error
    
    while not work.empty():
        a = work.get_nowait()
        
        # = = = = = = = = = = = = = = =
        
        crawler_status = 'error'
        for b in range(666):
            try:
                resp = requests.get(input_.loc[a, 'Description_Url'],
                                    #headers=UA.get_User_Agent_Requests(),
                                    proxies=Proxy.get_Proxy_Requests()).text
                
                # = = = = = = = = = = = = = = =
                
                soup = BeautifulSoup(resp, 'lxml')
                html = etree.HTML(str(soup))
                
                # = = = = = = = = = = = = = = =
                
                list_div = html.xpath('//div[@class="items__list--content"]')

                # = = = = = = = = = = = = = = =
                
                list_h5 = []
                for div in list_div:
                    if len(div.xpath('./h5/text()')) > 0:
                        list_h5.append(div.xpath('./h5/text()')[0])
                
                # = = = = = = = = = = = = = = =
                
                list_list_li = []
                for div in list_div:
                    list_li = div.xpath('./ul/li')
                    list_li = [li.xpath('./text()')[0] for li in list_li]
                    list_list_li.append(list_li)
    
                # = = = = = = = = = = = = = = =
                
                list_vehicle = []
                for h5, list_li in zip(list_h5, list_list_li):
                    list_li = [h5 + ' ' + li for li in list_li]
                    list_vehicle += list_li

                vehicle = '\n'.join(list_vehicle)

                # = = = = = = = = = = = = = = =
                
                df_temp = pd.DataFrame([{'Description_Url': input_.loc[a, 'Description_Url'],
                                         'Vehicle': vehicle}])

                output_correct = pd.concat([output_correct, df_temp], ignore_index=True).fillna('')

                # = = = = = = = = = = = = = = =

                crawler_status = 'ok'

                break
            except:
                time.sleep(0.3)
                continue
        
        # = = = = = = = = = = = = = = =
        
        if crawler_status == 'error':
            df_temp = pd.DataFrame([{'Description_Url': input_.loc[a, 'Description_Url']}])

            output_error = pd.concat([output_error, df_temp], ignore_index=True).fillna('')
            
        # = = = = = = = = = = = = = = =
        
        print('[' + crawler_status + '] - ' + input_.loc[a, 'Description_Url'] +  '\n[尝试次数：' + str(b+1) + '] - [剩余数量：' + str(work.qsize()) + '] - [当前时间：' + datetime.now().strftime('%H:%M:%S') + ']\n')
        
# = = = = = = = = = = = = = = =

list_task = []
for _ in range(30):
    task = gevent.spawn(crawler)
    list_task.append(task)
gevent.joinall(list_task)

print('输出ing...')
print()
output_correct = output_correct.drop_duplicates(ignore_index=True)
output_correct = output_correct.sort_values(by=['Description_Url'], ascending=[True]).reset_index(drop=True)
output_correct.to_excel('./description-' + datetime.now().strftime('%Y%m%d_%H%M%S') + '.xlsx', index=False)
if len(output_error) != 0:
    output_error = output_error.drop_duplicates(ignore_index=True)
    output_error = output_error.sort_values(by=['Description_Url'], ascending=[True]).reset_index(drop=True)
    output_error.to_excel('./description_error.xlsx', index=False)
    print('爬虫存在error')
    print()
print('Done ~')

总数量：3337

[ok] - https://vi.vipr.ebaydesc.com/itmdesc/352570922501?t=1715338404000&category=33710&seller=carpartswholesale&excSoj=1&excTrk=1&lsite=100&ittenable=true&domain=ebay.com&descgauge=1&cspheader=1&oneClk=2&secureDesc=1
[尝试次数：1] - [剩余数量：3307] - [当前时间：18:44:17]

[ok] - https://vi.vipr.ebaydesc.com/itmdesc/373651377645?t=1715425697000&category=33710&seller=carpartswholesale&excSoj=1&excTrk=1&lsite=100&ittenable=true&domain=ebay.com&descgauge=1&cspheader=1&oneClk=2&secureDesc=1
[尝试次数：1] - [剩余数量：3306] - [当前时间：18:44:17]

[ok] - https://vi.vipr.ebaydesc.com/itmdesc/353523599464?t=1715342652000&category=33710&seller=carpartswholesale&excSoj=1&excTrk=1&lsite=100&ittenable=true&domain=ebay.com&descgauge=1&cspheader=1&oneClk=2&secureDesc=1
[尝试次数：1] - [剩余数量：3305] - [当前时间：18:44:18]

[ok] - https://vi.vipr.ebaydesc.com/itmdesc/234039594910?t=1715262863000&category=33710&seller=carpartswholesale&excSoj=1&excTrk=1&lsite=100&ittenable=true&domain=ebay.com&descgauge=1&cspheader=1&oneClk=2&secur

[ok] - https://vi.vipr.ebaydesc.com/itmdesc/373603464928?t=1715423587000&category=33710&seller=carpartswholesale&excSoj=1&excTrk=1&lsite=100&ittenable=true&domain=ebay.com&descgauge=1&cspheader=1&oneClk=2&secureDesc=1
[尝试次数：1] - [剩余数量：3272] - [当前时间：18:44:20]

[ok] - https://vi.vipr.ebaydesc.com/itmdesc/353598072958?t=1715344795000&category=33710&seller=carpartswholesale&excSoj=1&excTrk=1&lsite=100&ittenable=true&domain=ebay.com&descgauge=1&cspheader=1&oneClk=2&secureDesc=1
[尝试次数：1] - [剩余数量：3271] - [当前时间：18:44:20]

[ok] - https://vi.vipr.ebaydesc.com/itmdesc/232995214584?t=1715255440000&category=33710&seller=carpartswholesale&excSoj=1&excTrk=1&lsite=100&ittenable=true&domain=ebay.com&descgauge=1&cspheader=1&oneClk=2&secureDesc=1
[尝试次数：1] - [剩余数量：3270] - [当前时间：18:44:20]

[ok] - https://vi.vipr.ebaydesc.com/itmdesc/353576966478?t=1715343682000&category=33710&seller=carpartswholesale&excSoj=1&excTrk=1&lsite=100&ittenable=true&domain=ebay.com&descgauge=1&cspheader=1&oneClk=2&secureDesc=1
[尝

[ok] - https://vi.vipr.ebaydesc.com/itmdesc/353577006295?t=1715343499000&category=33710&seller=carpartswholesale&excSoj=1&excTrk=1&lsite=100&ittenable=true&domain=ebay.com&descgauge=1&cspheader=1&oneClk=2&secureDesc=1
[尝试次数：1] - [剩余数量：3238] - [当前时间：18:44:23]

[ok] - https://vi.vipr.ebaydesc.com/itmdesc/352511404246?t=1715335508000&category=33710&seller=carpartswholesale&excSoj=1&excTrk=1&lsite=100&ittenable=true&domain=ebay.com&descgauge=1&cspheader=1&oneClk=2&secureDesc=1
[尝试次数：1] - [剩余数量：3237] - [当前时间：18:44:23]

[ok] - https://vi.vipr.ebaydesc.com/itmdesc/354763677013?t=1715348892000&category=33710&seller=carpartswholesale&excSoj=1&excTrk=1&lsite=100&ittenable=true&domain=ebay.com&descgauge=1&cspheader=1&oneClk=2&secureDesc=1
[尝试次数：1] - [剩余数量：3236] - [当前时间：18:44:23]

[ok] - https://vi.vipr.ebaydesc.com/itmdesc/232440803554?t=1715252992000&category=33710&seller=carpartswholesale&excSoj=1&excTrk=1&lsite=100&ittenable=true&domain=ebay.com&descgauge=1&cspheader=1&oneClk=2&secureDesc=1
[尝

[ok] - https://vi.vipr.ebaydesc.com/itmdesc/234999224759?t=1715271244000&category=33710&seller=carpartswholesale&excSoj=1&excTrk=1&lsite=100&ittenable=true&domain=ebay.com&descgauge=1&cspheader=1&oneClk=2&secureDesc=1
[尝试次数：3] - [剩余数量：3203] - [当前时间：18:44:27]

[ok] - https://vi.vipr.ebaydesc.com/itmdesc/233015774243?t=1715255732000&category=33710&seller=carpartswholesale&excSoj=1&excTrk=1&lsite=100&ittenable=true&domain=ebay.com&descgauge=1&cspheader=1&oneClk=2&secureDesc=1
[尝试次数：2] - [剩余数量：3202] - [当前时间：18:44:27]

[ok] - https://vi.vipr.ebaydesc.com/itmdesc/374678432042?t=1715432086000&category=33710&seller=carpartswholesale&excSoj=1&excTrk=1&lsite=100&ittenable=true&domain=ebay.com&descgauge=1&cspheader=1&oneClk=2&secureDesc=1
[尝试次数：8] - [剩余数量：3201] - [当前时间：18:44:27]

[ok] - https://vi.vipr.ebaydesc.com/itmdesc/233087159046?t=1715257952000&category=33710&seller=carpartswholesale&excSoj=1&excTrk=1&lsite=100&ittenable=true&domain=ebay.com&descgauge=1&cspheader=1&oneClk=2&secureDesc=1
[尝

[ok] - https://vi.vipr.ebaydesc.com/itmdesc/234091518655?t=1715264693000&category=33710&seller=carpartswholesale&excSoj=1&excTrk=1&lsite=100&ittenable=true&domain=ebay.com&descgauge=1&cspheader=1&oneClk=2&secureDesc=1
[尝试次数：1] - [剩余数量：3170] - [当前时间：18:44:33]

[ok] - https://vi.vipr.ebaydesc.com/itmdesc/355544552848?t=1715353938000&category=33710&seller=carpartswholesale&excSoj=1&excTrk=1&lsite=100&ittenable=true&domain=ebay.com&descgauge=1&cspheader=1&oneClk=2&secureDesc=1
[尝试次数：7] - [剩余数量：3169] - [当前时间：18:44:33]

[ok] - https://vi.vipr.ebaydesc.com/itmdesc/234091522149?t=1715264749000&category=33710&seller=carpartswholesale&excSoj=1&excTrk=1&lsite=100&ittenable=true&domain=ebay.com&descgauge=1&cspheader=1&oneClk=2&secureDesc=1
[尝试次数：8] - [剩余数量：3168] - [当前时间：18:44:33]

[ok] - https://vi.vipr.ebaydesc.com/itmdesc/235397361871?t=1715279434000&category=33710&seller=carpartswholesale&excSoj=1&excTrk=1&lsite=100&ittenable=true&domain=ebay.com&descgauge=1&cspheader=1&oneClk=2&secureDesc=1
[尝

...

[ok] - https://vi.vipr.ebaydesc.com/itmdesc/374759012559?t=1715432935000&category=33645&seller=carpartswholesale&excSoj=1&excTrk=1&lsite=100&ittenable=true&domain=ebay.com&descgauge=1&cspheader=1&oneClk=2&secureDesc=1
[尝试次数：5] - [剩余数量：120] - [当前时间：18:53:08]

[ok] - https://vi.vipr.ebaydesc.com/itmdesc/235123935034?t=1715273603000&category=33710&seller=carpartswholesale&excSoj=1&excTrk=1&lsite=100&ittenable=true&domain=ebay.com&descgauge=1&cspheader=1&oneClk=2&secureDesc=1
[尝试次数：1] - [剩余数量：119] - [当前时间：18:53:08]

[ok] - https://vi.vipr.ebaydesc.com/itmdesc/234944381376?t=1715270699000&category=33710&seller=carpartswholesale&excSoj=1&excTrk=1&lsite=100&ittenable=true&domain=ebay.com&descgauge=1&cspheader=1&oneClk=2&secureDesc=1
[尝试次数：8] - [剩余数量：118] - [当前时间：18:53:08]

[ok] - https://vi.vipr.ebaydesc.com/itmdesc/235042551562?t=1715271977000&category=33710&seller=carpartswholesale&excSoj=1&excTrk=1&lsite=100&ittenable=true&domain=ebay.com&descgauge=1&cspheader=1&oneClk=2&secureDesc=1
[尝试次数

[ok] - https://vi.vipr.ebaydesc.com/itmdesc/373603473117?t=1715423891000&category=33710&seller=carpartswholesale&excSoj=1&excTrk=1&lsite=100&ittenable=true&domain=ebay.com&descgauge=1&cspheader=1&oneClk=2&secureDesc=1
[尝试次数：4] - [剩余数量：88] - [当前时间：18:53:13]

[ok] - https://vi.vipr.ebaydesc.com/itmdesc/354973087427?t=1715350733000&category=33710&seller=carpartswholesale&excSoj=1&excTrk=1&lsite=100&ittenable=true&domain=ebay.com&descgauge=1&cspheader=1&oneClk=2&secureDesc=1
[尝试次数：2] - [剩余数量：87] - [当前时间：18:53:13]

[ok] - https://vi.vipr.ebaydesc.com/itmdesc/374728458909?t=1715432540000&category=33645&seller=carpartswholesale&excSoj=1&excTrk=1&lsite=100&ittenable=true&domain=ebay.com&descgauge=1&cspheader=1&oneClk=2&secureDesc=1
[尝试次数：1] - [剩余数量：86] - [当前时间：18:53:13]

[ok] - https://vi.vipr.ebaydesc.com/itmdesc/355061702092?t=1715351484000&category=33710&seller=carpartswholesale&excSoj=1&excTrk=1&lsite=100&ittenable=true&domain=ebay.com&descgauge=1&cspheader=1&oneClk=2&secureDesc=1
[尝试次数：20

[ok] - https://vi.vipr.ebaydesc.com/itmdesc/355102682006?t=1715351682000&category=33710&seller=carpartswholesale&excSoj=1&excTrk=1&lsite=100&ittenable=true&domain=ebay.com&descgauge=1&cspheader=1&oneClk=2&secureDesc=1
[尝试次数：1] - [剩余数量：55] - [当前时间：18:53:18]

[ok] - https://vi.vipr.ebaydesc.com/itmdesc/235227490881?t=1715275016000&category=33710&seller=carpartswholesale&excSoj=1&excTrk=1&lsite=100&ittenable=true&domain=ebay.com&descgauge=1&cspheader=1&oneClk=2&secureDesc=1
[尝试次数：3] - [剩余数量：54] - [当前时间：18:53:19]

[ok] - https://vi.vipr.ebaydesc.com/itmdesc/235058315755?t=1715272368000&category=33710&seller=carpartswholesale&excSoj=1&excTrk=1&lsite=100&ittenable=true&domain=ebay.com&descgauge=1&cspheader=1&oneClk=2&secureDesc=1
[尝试次数：2] - [剩余数量：53] - [当前时间：18:53:19]

[ok] - https://vi.vipr.ebaydesc.com/itmdesc/235310813556?t=1715276527000&category=33710&seller=carpartswholesale&excSoj=1&excTrk=1&lsite=100&ittenable=true&domain=ebay.com&descgauge=1&cspheader=1&oneClk=2&secureDesc=1
[尝试次数：9]

[ok] - https://vi.vipr.ebaydesc.com/itmdesc/374952597093?t=1715434688000&category=33710&seller=carpartswholesale&excSoj=1&excTrk=1&lsite=100&ittenable=true&domain=ebay.com&descgauge=1&cspheader=1&oneClk=2&secureDesc=1
[尝试次数：2] - [剩余数量：23] - [当前时间：18:53:25]

[ok] - https://vi.vipr.ebaydesc.com/itmdesc/354728648235?t=1715348655000&category=33710&seller=carpartswholesale&excSoj=1&excTrk=1&lsite=100&ittenable=true&domain=ebay.com&descgauge=1&cspheader=1&oneClk=2&secureDesc=1
[尝试次数：3] - [剩余数量：22] - [当前时间：18:53:25]

[ok] - https://vi.vipr.ebaydesc.com/itmdesc/374952593241?t=1715434648000&category=33710&seller=carpartswholesale&excSoj=1&excTrk=1&lsite=100&ittenable=true&domain=ebay.com&descgauge=1&cspheader=1&oneClk=2&secureDesc=1
[尝试次数：4] - [剩余数量：21] - [当前时间：18:53:25]

[ok] - https://vi.vipr.ebaydesc.com/itmdesc/235123931820?t=1715273569000&category=33710&seller=carpartswholesale&excSoj=1&excTrk=1&lsite=100&ittenable=true&domain=ebay.com&descgauge=1&cspheader=1&oneClk=2&secureDesc=1
[尝试次数：1]

[ok] - https://vi.vipr.ebaydesc.com/itmdesc/235104872173?t=1715273045000&category=33644&seller=carpartswholesale&excSoj=1&excTrk=1&lsite=100&ittenable=true&domain=ebay.com&descgauge=1&cspheader=1&oneClk=2&secureDesc=1
[尝试次数：1] - [剩余数量：0] - [当前时间：18:53:29]

[ok] - https://vi.vipr.ebaydesc.com/itmdesc/235005362287?t=1715271462000&category=33710&seller=carpartswholesale&excSoj=1&excTrk=1&lsite=100&ittenable=true&domain=ebay.com&descgauge=1&cspheader=1&oneClk=2&secureDesc=1
[尝试次数：10] - [剩余数量：0] - [当前时间：18:53:30]

[ok] - https://vi.vipr.ebaydesc.com/itmdesc/374640950719?t=1715431902000&category=33710&seller=carpartswholesale&excSoj=1&excTrk=1&lsite=100&ittenable=true&domain=ebay.com&descgauge=1&cspheader=1&oneClk=2&secureDesc=1
[尝试次数：4] - [剩余数量：0] - [当前时间：18:53:30]

[ok] - https://vi.vipr.ebaydesc.com/itmdesc/354728648227?t=1715348654000&category=33710&seller=carpartswholesale&excSoj=1&excTrk=1&lsite=100&ittenable=true&domain=ebay.com&descgauge=1&cspheader=1&oneClk=2&secureDesc=1
[尝试次数：5] -